In [1]:
import os
import json
import openai
import requests
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt


GPT_MODEL = "gpt-4-0613"

load_dotenv(".env")
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [3]:
class Chat:
    def __init__(self):
        self.conversation_history = []

    def add_prompt(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self):
        for message in self.conversation_history:
            print(
                f"{message['role']}: {message['content']}\n\n",
                message["role"],
            )


In [4]:
functions = [
    {
        "name": "save_users_questionire",
        "description": "If user responded all questiones, store fully filled questionire to the database",
        "parameters": {
            "type": "object",
            "properties": {
                "user_answers": {
                    "type": "object",
                    "description": "Keys of the dict are questions to the user and values are user's responses to the coresponding questions",
                },
            },
            "required": ["user_answers"],
        },
    },
    {
        "name": "ask_follow_up_question",
        "description": "If the user didn't answer all the questions, generates an additional question to ask user.",
        "parameters": {
            "type": "object",
            "properties": {
                "next_question": {
                    "type": "string",
                    "description": "Next question which we will ask user to clarify their response",
                },
            },
            "required": ["next_question"],
        },
    },
]

In [5]:
chat = Chat()

In [6]:
chat.add_prompt("system", 
    """
    You are a polite and smart AI assistant that helps people to fill questionire to apply for an insurance.
    We need to fill next questions:
    1) What is your first name?
    2) What is your last name?
    3) What is the type of insurance you need?
    4) What is your phone number?
    5) What is your age?

    We expect final response in json format with keys: "first_name", "last_name", "age", "type_of_insurance", "phone_number". 

    Allowed types of insurance are: "Auto", "Home", "Condo", "Tenant", "Farm", "Commercial", "Life".
    
    Make sure that the phone number either has 10 digits or (11 digits and starts with +1). 
    Don't save +1 for the phone number, we need only next 10 digits. Store as int.
    
    Age should be int value with year granularity, don't accept a string.    
    
    Please ask one question at a time.
    """)

In [7]:
chat.add_prompt("user", 
   """
   Hi, I'm Bob Smith. I'm looking for a car insurance. Do you offer it?

   """)

In [8]:
chat_response = chat_completion_request(
        chat.conversation_history,
        functions=functions
    )


In [9]:
chat_response

<Response [200]>

In [10]:
chat_response.__dict__

{'_content': b'{\n  "id": "chatcmpl-7dUqA0SHBxPWXa7m1kZeMvSYwTiJO",\n  "object": "chat.completion",\n  "created": 1689647698,\n  "model": "gpt-4-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "Yes, we do offer car insurance. Can I have your phone number, please?"\n      },\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 344,\n    "completion_tokens": 18,\n    "total_tokens": 362\n  }\n}\n',
 '_content_consumed': True,
 '_next': None,
 'status_code': 200,
 'headers': {'Date': 'Tue, 18 Jul 2023 02:35:00 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'Cache-Control': 'no-cache, must-revalidate', 'openai-model': 'gpt-4-0613', 'openai-organization': 'user-zqtgiixglanxzg0gvijsewh3', 'openai-processing-ms': '2189', 'openai-version': '2020-10-01', 'strict-transport-security': 'max-age=15724800; include

In [18]:
message = chat_response.json()['choices'][0]['message']
message

{'role': 'assistant',
 'content': 'Yes, we do offer car insurance. Can I have your phone number, please?'}

In [19]:
chat.add_prompt(message['role'], message['content'])

In [21]:
chat.add_prompt("user", 
   """
   Yes, it's +19876543210.

   """)

In [22]:
chat_response = chat_completion_request(
        chat.conversation_history,
        functions=functions
    )
chat_response.__dict__

{'_content': b'{\n  "id": "chatcmpl-7dUzfegeY45lEk06wwFpE3v1hr4Az",\n  "object": "chat.completion",\n  "created": 1689648287,\n  "model": "gpt-4-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "Great, thank you for that. Could you please tell me your age?"\n      },\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 384,\n    "completion_tokens": 16,\n    "total_tokens": 400\n  }\n}\n',
 '_content_consumed': True,
 '_next': None,
 'status_code': 200,
 'headers': {'Date': 'Tue, 18 Jul 2023 02:44:50 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'Cache-Control': 'no-cache, must-revalidate', 'openai-model': 'gpt-4-0613', 'openai-organization': 'user-zqtgiixglanxzg0gvijsewh3', 'openai-processing-ms': '2652', 'openai-version': '2020-10-01', 'strict-transport-security': 'max-age=15724800; includeSubDomai

In [23]:
message = chat_response.json()['choices'][0]['message']
message

{'role': 'assistant',
 'content': 'Great, thank you for that. Could you please tell me your age?'}

In [24]:
chat.add_prompt(message['role'], message['content'])

In [25]:
chat.add_prompt("user", 
   """
   I'm under 30.

   """)

In [26]:
chat_response = chat_completion_request(
        chat.conversation_history,
        functions=functions
    )
chat_response.__dict__

{'_content': b'{\n  "id": "chatcmpl-7dV0hTGaUTVhKTjpfSRFwFNc0HN6y",\n  "object": "chat.completion",\n  "created": 1689648351,\n  "model": "gpt-4-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": null,\n        "function_call": {\n          "name": "ask_follow_up_question",\n          "arguments": "\\n{\\n  \\"next_question\\": \\"Could you please specify your exact age?\\"\\n}"\n        }\n      },\n      "finish_reason": "function_call"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 416,\n    "completion_tokens": 25,\n    "total_tokens": 441\n  }\n}\n',
 '_content_consumed': True,
 '_next': None,
 'status_code': 200,
 'headers': {'Date': 'Tue, 18 Jul 2023 02:45:54 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'Cache-Control': 'no-cache, must-revalidate', 'openai-model': 'gpt-4-0613', 'openai-organization': 'user-zqtgiixglanx

In [29]:
message = chat_response.json()['choices'][0]["message"]
message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'ask_follow_up_question',
  'arguments': '\n{\n  "next_question": "Could you please specify your exact age?"\n}'}}

In [35]:
import json

In [37]:
chat_response.json()['choices'][0]["message"]['function_call']['arguments']

'\n{\n  "next_question": "Could you please specify your exact age?"\n}'

In [39]:
next_question = json.loads(
    chat_response.json()['choices'][0]["message"]['function_call']['arguments']
    )['next_question']

message = {'role': 'assistant',
           'content': next_question}

message

{'role': 'assistant', 'content': 'Could you please specify your exact age?'}

In [40]:
chat.add_prompt(message['role'], message['content'])

In [41]:
chat.add_prompt("user", 
   """
   I'm 25.

   """)

In [42]:
chat_response = chat_completion_request(
        chat.conversation_history,
        functions=functions
    )
chat_response.__dict__

{'_content': b'{\n  "id": "chatcmpl-7dV8lXuXLp4ekQChtxK8QsZITCAID",\n  "object": "chat.completion",\n  "created": 1689648851,\n  "model": "gpt-4-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": null,\n        "function_call": {\n          "name": "save_users_questionire",\n          "arguments": "\\n{\\n\\"first_name\\": \\"Bob\\",\\n\\"last_name\\": \\"Smith\\",\\n\\"age\\": 25,\\n\\"type_of_insurance\\": \\"Auto\\",\\n\\"phone_number\\": 9876543210\\n}"\n        }\n      },\n      "finish_reason": "function_call"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 440,\n    "completion_tokens": 49,\n    "total_tokens": 489\n  }\n}\n',
 '_content_consumed': True,
 '_next': None,
 'status_code': 200,
 'headers': {'Date': 'Tue, 18 Jul 2023 02:54:17 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'Cache-Control': 'no-cache, must-reva

In [43]:
chat_response.json()['choices'][0]["message"]['function_call']['arguments']

'\n{\n"first_name": "Bob",\n"last_name": "Smith",\n"age": 25,\n"type_of_insurance": "Auto",\n"phone_number": 9876543210\n}'

In [44]:
next_question = json.loads(
    chat_response.json()['choices'][0]["message"]['function_call']['arguments']
    )

next_question

{'first_name': 'Bob',
 'last_name': 'Smith',
 'age': 25,
 'type_of_insurance': 'Auto',
 'phone_number': 9876543210}

In [46]:
SYSTEM_SETUP_PROMPT = \
    """
    You are a polite and smart AI assistant that helps people to fill questionire to apply for an insurance.
    We need to fill next questions:
    1) What is your first name?
    2) What is your last name?
    3) What is the type of insurance you need?
    4) What is your phone number?
    5) What is your age?

    We expect final response in json format with keys: "first_name", "last_name", "age", "type_of_insurance", "phone_number". 

    Allowed types of insurance are: "Auto", "Home", "Condo", "Tenant", "Farm", "Commercial", "Life".
    
    Make sure that the phone number either has 10 digits or (11 digits and starts with +1). 
    Don't save +1 for the phone number, we need only next 10 digits. Store as int.
    
    Age should be int value with year granularity, don't accept a string.    
    
    Please ask one question at a time.
    """

In [47]:
FUNCTIONS = [
    {
        "name": "save_users_questionire",
        "description": "If user responded all questiones, store fully filled questionire to the database",
        "parameters": {
            "type": "object",
            "properties": {
                "user_answers": {
                    "type": "object",
                    "description": "Keys of the dict are questions to the user and values are user's responses to the coresponding questions",
                },
            },
            "required": ["user_answers"],
        },
    },
    {
        "name": "ask_follow_up_question",
        "description": "If the user didn't answer all the questions, generates an additional question to ask user.",
        "parameters": {
            "type": "object",
            "properties": {
                "next_question": {
                    "type": "string",
                    "description": "Next question which we will ask user to clarify their response",
                },
            },
            "required": ["next_question"],
        },
    },
]

In [ ]:
def generate_response(chat: Chat, user_message: str, functions: list = FUNCTIONS
                    ) -> bool, str:
    
    chat_response = chat_completion_request(
        chat.conversation_history,
        functions=functions
    )
    
    response_content

In [ ]:
chat = Chat()

chat.add_prompt("system", SYSTEM_SETUP_PROMPT)

chat_finished = True

for i in range(2):
    response_content = chat_completion_request(
        chat.conversation_history,
        functions=functions
    )

    if chat_response is not None:
        response_content = chat_response.json()['choices'][0]['message']

        print(response_content)
        
        message = chat_response.json()['choices'][0]['message']
        
        print("message:", message)
        
        if message is not None:
            chat_finished = False
            
            return chat_finished, message
        

        if 'function_call' in response_content:
            if response_content['function_call']['name'] == 'save_users_questionire':

                print(response_content)
                questionire = json.loads(response_content['function_call']['arguments'])
                print("Result questionire:")
                print(questionire)
                
                chat_finished = True
                return chat_finished, questionire
            
            elif response_content['function_call']['name'] == 'ask_follow_up_question':
                print(response_content)
                next_question = json.loads(response_content['function_call']['arguments'])['next_question']
                print("Next question:")
                print(next_question)
                
                chat_finished = False
                return chat_finished, next_question           
        
    else:
        print("ChatCompletion request failed. Retrying...")